In [1]:
from noise_dataset import NoiseDataset # importando nosso dataset de ruido
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from math import floor # para dividir dataset

In [9]:
from network_bool import NN_b

In [2]:
#verificando dispositivo a ser utilizado
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo {device}")

Usando o dispositivo cpu


In [47]:
# criando nosso calculador de acuracia
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad(): # usamos no_grad para nao alterar parametros da rede com cada chamada.
    for data, target in loader:
      data = data.to(device)
      target = target.to(device)
      target = target.unsqueeze(1)
      scores = model(data)
      scores[scores > 0.5] =1 
      scores[scores < 0.5] = 0      
      predictions = scores
      # print(target)
      # print(predictions)
      # print(predictions == target)
      num_correct += (predictions == target).sum()
      num_samples += predictions.shape[0]
  
  model.train()
  return num_correct / num_samples


In [4]:
# definindo variaveis importantes
operator_size = 5
learning_rate = 0.01
batch_size = 12
num_epochs = 4

In [5]:
# carregando nosso dataset
dataset = NoiseDataset()
size = len(dataset)
train, test = torch.utils.data.random_split(dataset, lengths=[floor(0.7*size), floor(0.3*size)])
train_data = DataLoader(train, batch_size=batch_size, shuffle=True)
test_data = DataLoader(test, batch_size=batch_size, shuffle=True)

In [10]:
model = NN_b().to(device)

In [57]:
# criando loss function e otimizador
loss = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([6.5])) 
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [58]:
# treinando nossa rede
for epoch in range(num_epochs):
  for i, (data, target) in enumerate(train_data):
    # definindo tensores para CUDA:
    data = data.to(device)
    target = target.to(device).unsqueeze(1).float()
  
    preds = model(data)
    error = loss(preds, target)
		
    optimizer.zero_grad() # zerando gradiente para atualizar logo em seguida
    error.backward()
    optimizer.step()

In [59]:
check_accuracy(test_data, model)

tensor(0.9730)

In [60]:
def check_error(loader, model, loss):
  num_correct = 0
  num_samples = 0
  model.eval()
  running_error=0.0
  with torch.no_grad(): # usamos no_grad para nao alterar parametros da rede com cada chamada.
    for data, target in loader:
      data = data.to(device)
      target = target.to(device).unsqueeze(1).float()
      scores = model(data)        
      num_samples += scores.shape[0]
      running_error += loss(scores, target).item()
  model.train()
  return running_error/num_samples

In [61]:
check_error(test_data, model, loss)

0.01262980917270761

In [62]:
torch.save(model.state_dict(), 'model_bool.st')